In [ ]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import warnings
from sodapy import Socrata
import datetime
_default_date_format = "%Y-%m-%d"
dt = datetime
def date_to_int(date_string: str, form: str=_default_date_format) -> int:
    """Return date date_string in format form as an integer"""
    return dt.datetime.strptime(date_string, form).toordinal()

def int_to_date(ordinal: int, form: str=_default_date_format) -> str:
    """Return the day number ordinal to as a string, formatted with form"""
    return dt.datetime.fromordinal(ordinal).strftime(form)
def next_day(date):
    return int_to_date(date_to_int(date) + 1)

def date_to_index(date_string):
    df = covid_state_data_normalized["LA"]
    for n in df.index:
        if df.date.loc[n] >= date_string:
            return n
    raise ValueError("Date out of range")
    
state_populations = pd.read_csv("../data/input/state_populations_2019.csv")

In [ ]:
# Download data from CDC and preprocess

dataframe_list = [] # Downloaded dataframes go into this list

# COVID-19 cases
client = Socrata("data.cdc.gov",
                "YeLTGbOCV2gjoCenq2ve5LzDm")
for record_identifier in ["9mfq-cb36"]:#, "unsk-b7fc"]:
    results = client.get(record_identifier, limit=50000)
    df = pd.DataFrame.from_records(results)
    dataframe_list.append(df)

# COVID-19 hospitalizations
client = Socrata("healthdata.gov",
                "YeLTGbOCV2gjoCenq2ve5LzDm")
results = client.get("g62h-syeh", limit=50000)
df = pd.DataFrame.from_records(results)
dataframe_list.append(df)

# We only want a few columns from each dataset
pruned_df_list = []

df = dataframe_list[0]
df_pruned = df[["submission_date","state","new_case"]]
pruned_df_list.append(df_pruned.rename({"submission_date":"date",
                                       "new_case":"cases"}, axis=1))

df = dataframe_list[1]
df_pruned = df[["date","state","inpatient_beds_used_covid"]]
pruned_df_list.append(df_pruned.rename({"inpatient_beds_used_covid":"beds"},
                                       axis=1))

# Include only date, not time, in date column
for df in pruned_df_list:
    for n in df.date.index:
        df.date[n] = df.date[n].split("T")[0]

# Start data on 2020-04-01, end datasets at the earliest
# end date of the two datasets (they are sometimes off by one)
start_date = "2020-04-01"
end_date = min(df.date.max() for df in pruned_df_list)

# 50 states plus Washington DC and Puerto Rico
state_list = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
              'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA',
              'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE',
              'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR',
              'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI',
              'WV', 'WY']

# Separate data by state and save to .csv files
covid_state = {}
for state in state_list:
    for n in [1,0]:
        df = pruned_df_list[n]
        df_state = df.loc[df.state == state].copy()
        df_state.sort_values("date", inplace=True)
        df_state = df_state.loc[(df_state.date>=start_date) & \
                               (df_state.date<=end_date)].copy()
        if n == 1:
            df_all = df_state[["date"]].reset_index(drop=True).copy()
            df_all["beds"] = df_state["beds"].values.astype(np.float)
        elif n == 0:
            df_all["cases"] = df_state.cases.values.astype(np.float)

    L = []
    
    # Add 7-day average case numbers
    for n in range(len(df_all)):
        if n < 6:
            L.append(np.mean(df_all.cases.loc[:n]))
        else:
            L.append(np.mean(df_all.cases.loc[n-6:n]))
    df_all["cases_7day"] = L

    covid_state[state] = df_all[["date","beds","cases_7day"]].copy()

covid_state_data_normalized = {}

# Normalize by population
for state in state_list:
    population = state_populations[state][0]
    covid_state_data_normalized[state] = covid_state[state].copy()
    for column in ["beds", "cases_7day"]:
        normalized_data = covid_state[state][column]/population*10**5
        covid_state_data_normalized[state][column] = normalized_data.copy()

# Trim outliers
for state in state_list:
    df = covid_state_data_normalized[state]
    series = df.beds.copy()
    for n in range(1,len(df)-1):
        a,b = df.iloc[n-1].beds, df.iloc[n+1].beds
        MIN, MAX = min(a,b), max(a,b)
        if not (MIN*0.6 < df.iloc[n].beds < MAX*1.4):
            series.iloc[n] = (a+b)/2
    df.beds = series
    
# Add lag variables        
for state in state_list:
    for s in ["beds","cases_7day",]:
        for f in range(1,15):
            covid_state_data_normalized[state][f"{s}_{f}"] = \
                covid_state_data_normalized[state][s].shift(f)
    covid_state_data_normalized[state] = covid_state_data_normalized[state].copy()

In [ ]:
%%time
# Compute range of ARIMA predictions for each state
# These predictions are used to train Ridge model

def get_arima_forecast(state):
    df = covid_state_data_normalized[state]
    Q = {}
    for n in range(1,15):
        Q[f"arima_{n}"] = pd.Series(index=df.index, dtype='float64')
#         np.zeros(validation_end_date - validation_start_date + 1)
    for begin in range(validation_start_date-14, validation_end_date):
        with warnings.catch_warnings():
            success = False
            n = 50
            while not success:
                try:
                    warnings.filterwarnings("ignore")
                    model = ARIMA(df.loc[n:begin].beds, order = (5,2,2))
                    model_fit = model.fit()
                    predicted = model_fit.forecast(14)
                    if predicted.max() > df.loc[begin].beds * 4: # Some sort of numerical error
                        raise ValueError()
                    success = True
                except ValueError:
                    # ARIMA.fit sometimes has difficulty with L/U
                    # decomposition. If this happens, adjust the training
                    # window and try again.
                    n += 1
        for n in range(14):
            N = n+1
            if True or validation_start_date <= begin+N <= validation_end_date:
                Q[f"arima_{N}"][begin+N] = predicted.iloc[n]
    return Q, state

validation_end_date = date_to_index(covid_state_data_normalized["OR"].date.iloc[-1])
validation_start_date = validation_end_date - 40

start, stop = validation_start_date, validation_end_date

print("Computing all ARIMA predictions for each state (this may take a while)")
print("Completed: ", end='')
with Pool(3) as pool: # Each process tops out at around 500% CPU usage
    for Q, state in pool.imap_unordered(get_arima_forecast, state_list):
        df = covid_state_data_normalized[state].copy()
        Q_df = pd.DataFrame(Q)
        df[Q_df.columns] = Q_df * (Q_df>0) # Make sure it's non-negative
        covid_state_data_normalized[state] = df.copy()
        print(f"{state}, ", end='')
print()

# Uncomment to back up computations

# for state in state_list:
#     covid_state_data_normalized[state].to_csv(f"bak/{state}.csv",
#                                              index=False)

In [ ]:
state_populations[state][0]

In [ ]:
# Fit our models and make predictions

validation_end_date = date_to_index(covid_state_data_normalized["OR"].date.iloc[-1])
validation_start_date = validation_end_date - 25

start, stop = validation_start_date, validation_end_date
pred = {state:[] for state in state_list}
for state in state_list:
    df = covid_state_data_normalized[state]
    with warnings.catch_warnings():
        success = False
        n = 50
        while not success:
            try:
                warnings.filterwarnings("ignore")
                model = ARIMA(df.loc[n:].beds, order = (5,2,2))
                model_fit = model.fit()
                predicted = model_fit.forecast(14)
                if predicted.max() > df.iloc[-1].beds * 4: # Some sort of numerical error
                    raise ValueError()
                success = True
            except ValueError:
                # ARIMA.fit sometimes has difficulty with L/U
                # decomposition. If this happens, adjust the training
                # window and try again.
                n += 1
    df = df.loc[start:]
    for future in range(1,15):
        reg = Pipeline([('scaler', StandardScaler()),
                        ('ridge', Ridge(alpha=10))])
        reg.fit(df[[f"beds_{future}",
                    f"cases_7day_{future}",
                   f"arima_{future}"]].values,
               df.beds)
        if False and future == 1 and state == "AK":
            print((state_populations[state][0]/10**5*df[[f"beds_{future}",
                    f"cases_7day_{future}",
                   f"arima_{future}"]].values ,
               df.beds* state_populations[state]/10**5))
        arima_prediction = predicted.iloc[future-1]
        linear_prediction = reg.predict(np.concatenate((df.iloc[-1][["beds","cases_7day"]],
                              [arima_prediction])).reshape(1,-1))[0]
        baseline_prediction = df.beds.iloc[-1]
        prediction = np.mean([arima_prediction,
                             linear_prediction,
                             baseline_prediction])
        pred[state].append(prediction)
        if state == "AK":
            pop = state_populations[state][0] / 10**5
            print(pop*arima_prediction,pop*linear_prediction, pop*baseline_prediction)
        
for state in state_list:
    pred[state] = np.array(pred[state]) * state_populations[state][0] / 10**5

In [ ]:
covid_state_data_normalized["AK"].arima_4.loc[606] * state_populations["AK"] / 10**5

In [ ]:
# Function for generating graphs with predictions

def show_prediction(state):
    actual = covid_state_data_normalized[state].beds.iloc[-22:] \
                        * state_populations[state][0] / 10**5
    predicted = np.concatenate(([actual.iloc[-1]],pred[state]))
#     plt.figure(figsize=(12,9))
    plt.figure(figsize=(6,6))
    plt.plot(range(len(actual)),actual,
             label="Observed",
            c="blue")
    plt.plot(range(len(actual)-1,len(actual)+len(predicted)-1),
             predicted,
             '--',
             label="Predicted",
            c="red")
    plt.plot([len(actual)-1], [actual.iloc[-1]],'o',
             c="black",markersize=8)
    plt.text(len(actual)+.4,actual.iloc[-1]*0.99,
             str(int(actual.iloc[-1])),
            fontsize=16,
            color="black")
    plt.legend()
    plt.xlabel("Date", fontsize=16)
    plt.ylabel("Beds in use", fontsize=16)

    # Add a few dates as labels to x-axis
    x_pos = []
    x_date = []
    end_date = covid_state_data_normalized[state].date.iloc[-1]
    for k in range(-3,3):  
        new_date = datetime.datetime.strftime(datetime.datetime.strptime(end_date,
                                                                         "%Y-%m-%d")\
                                              + datetime.timedelta(days=7*k),
                                                          "%Y-%m-%d")
        new_pos = len(actual)-1+7*k
        if k == 0:
            new_date += "\n(most recent\nobservation)"
        x_pos.append(new_pos)
        x_date.append(new_date[5:])
    plt.xticks(x_pos, x_date, fontsize=12)
    title_string = f"COVID-19 hospitalizations in {state}"
    plt.title(title_string, fontsize=22)
    
    date = covid_state_data_normalized[state].date.iloc[-1]
    print(f"Predictions for COVID-19 hospitalizations in {state}")
    for n in range(14):
        date = next_day(date)
        prediction = np.round(pred[state][n],1)
        print(f"{date}: {prediction}")
    plt.savefig(f"../graphs/{state}.png")
    plt.show()
    plt.close()

In [ ]:
# Check out our predictions
for state in state_list:
    show_prediction(state)

In [ ]:
from os import getcwd

In [ ]:
getcwd()